# Data For Teams
> "Better than excel way to do team analytics"

- author: Dmytro Karabash, Maxim Korotkov 
- categories: [management, python]
- image: images/analysis-1841158_1920.jpg
- permalink: /fastreport/

## Introduction

You’re the boss now. You have a team or a business unit doing consultancy. Probably a bunch of Consultants helping clients, some Project Managers leading your projects in a nice Agile or whatever way. Everyone fills their timesheets (and you cut their bonuses if they don’t) – time is billed to clients and you have Fixed Fee projects also. Maybe smaller teams being parts of your team also – you can lead ten, hundred or thousand people (hey, dude – you run a unit of 1000 employees? Don’t read this article – hire someone who did) You probably even have an ERP or something and a CFO in a nice corner office. Do you have a good way to estimate how much profit every team member and project brings and ways to predict it with good precision? If you have a nice ERP dashboard which gives you all then – then you’re lucky and you don’t need this article either. But high chances are that you get a simple calculation end of year – like “let’s take all revenue finance recognized on your projects in a year and man day cost at \\$800 to get your unit profitability” (\\$800 can seem crazily high or unacceptably low, depending on business you’re in). So that’s the precision you have to the money your team brings to firm? Do we need to remind you that this is the only reason why your job exists?

What you probably have is – all the timesheets, projects with their budget and you can even get to approximate costs nicely (we’ll get there). Shitload of data for a year even for a team of 10. What I will tell you here is that you don’t need an ERP to do the rest – the notebook in Jupyter will do for now. Let’s remember - management starts with measurement. How can you supervise your team and projects if you don’t have their data at your fingers?

Why do you need Data Science here? Well, you can have all data an your fingers. Let me show you 3 Project Managers which have highest profit margin from beginning of the year - based on timesheets, revenue type and cost rates in one (not-so-simple) line

In [142]:
#TODO: Dmytro, we need a simple calculation here

In [143]:
#hide

#Note: we might need an internal project to “fill” the difference to full month (e.g. “INTERNAL” with revenue rate of 0 covering everything to full week)

## Data Science

So, let’s get this done. First – yes, you need to know a little bit of python to get the files in. Basic level will do. (If you are a manager in 2020 and can’t write a simple script – hm, you probably missed something). The objective is to have an easy-to-use toolkit. We are definitely not developing any ERP here. What you will see is a very simple set of tools - you spend an hour or two to load your files into the notebook and then start playing with it - just make the analysis you want. You can end up checking a few numbers or building your own charts and dashboards. It is all under L-GPL - free to use, even if you work for a corporation (that’s where you need it most) - just install Anaconda and download the notebook. 
So, we first get and transform our input files (if you want to follow text AND all the code - read it on cohab)

We are loading projects, timesheets and roles below - more details will follow on what we presumed is inside.

In [144]:
#hide
##loading functions

import pandas as pd

#multiple outputs (from ​​​​28 Jupyter Notebook Tips, Tricks, and Shortcuts)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

verbosity=5



In [145]:
#TODO:delete
# data = {}
# for csv_name in ['projects','timesheet', 'roles']:
#     url = 'https://raw.githubusercontent.com/h17/fastreport/master/data/{}.csv'.format(csv_name)
#     data[csv_name] = df = pd.read_csv(url, index_col=0)

In [146]:
data = {}
url = 'https://raw.githubusercontent.com/h17/fastreport/master/data/roles.csv'
data['roles'] = pd.read_csv(url, index_col=0,sep=";")

project_data = pd.ExcelFile('../data/project_data.xlsm')
project_data.sheet_names

timesheets = []
for sheet in project_data.sheet_names:
    if 'TS' in sheet:
        timesheets += [pd.read_excel(project_data,sheet,header=2)]
    else:
        tmp = pd.read_excel(project_data,sheet)
        data[sheet] = tmp 
if 'Sat.1' in timesheets[0]: timesheets[0] = timesheets[0].rename(columns={'Sat': 'Sun', 'Sat.1' : 'Sat'}) 
data['timesheet'] = pd.concat(timesheets,sort=False)
data['timesheet'] = data['timesheet'][~data['timesheet'].User.isnull()]
data.keys()

['TS Feb', 'TS Mar', 'TS Apr', 'Entity Project', 'HC']

dict_keys(['roles', 'Entity Project', 'HC', 'timesheet'])

In [147]:
data.keys()
for key in data.keys():
    print("{:20} shape: {:^10} rows \t and {:^10} columns".format(key, *data[key].shape))
    #data[key].head(5)

dict_keys(['roles', 'Entity Project', 'HC', 'timesheet'])

roles                shape:     9      rows 	 and     5      columns
Entity Project       shape:     48     rows 	 and     15     columns
HC                   shape:     35     rows 	 and     6      columns
timesheet            shape:    1792    rows 	 and     14     columns


In [148]:
#hide

#todo:separate

## Data we loaded

Let me summarize it here for you – in any decent firm, if you are a team leader or business unit manager you are involved in merit processes – you can get
* Dictionaries
    * Cost per region or default cost for “outside” contributors
    * Average revenue per hour for projects not “owned” by you
    

In [149]:
# show roles
roles=data["roles"]
#here we'd need to stack it on last 3 countries, for now I've used "EU" but that's wrong
roles.head()
def_revenue = 1200
def_cost = 850

,Position,Level,EU,UK,CH
DEFAULT COSTS per ROLE and REGION (in $),,,,,
DIRECTOR,DIR,SENIOR,1100,1300,1100
Program Manager,PRA,SEASONED,900,1000,900
Project Manager,PRO,SENIOR,800,800,800
Project Coordinator,PRO,SEASONED,700,700,700
Solution Architect,CON,EXPERT,900,1000,1050


* HeadCount
    * Graded costs
    * Engagement model (employee or contractor)

In [150]:
#show head count
headcount=data["HC"]
# todo: use stacked column instead of EU
headcount = headcount.merge(roles[['Position','Level','EU']],how='left',left_on=['Level','Position'],right_on=['Level','Position'])
headcount.head(n=5)

,NAME,TYPE,Location,Position,Level,ID,EU
0,JANE DOE,STAFF,CH,CON,SEASONED,1001,550
1,JOHN DOE,STAFF,PL,CON,JUNIOR,1002,500
2,JOHN SMITH,STAFF,CH,CON,SEASONED,1003,550
3,JACK THE MANAGER,CONTRACTOR,CH,PRO,SENIOR,1004,800
4,JOHN THE MANAGER,CONTRACTOR,CH,PRO,SENIOR,1005,800


* Projects
    * Budget, effort estimate, deadline
    * Project type (e.g. “On demand support” or “Subscription” will not have any actual deadline to control) - though it is ok if you don’t have it
    * Type of revenue recognition (Time and Material or Fixed Fee)


In [21]:
# show projects
projects = data["Entity Project"]
projects.head(n=5)

,Client ID,Project ID,Project,PM,Client Region,Start,End,Funding,BASE RATE USD,TCV,Recognized\nUSD,2017 effort,2018 effort,2019 effort,Jan 2020 Effort
0,13154,68454,Medium consutancy 1,SUSAN THE MANAGER,Europe,2019-01-16 08:00:00,2020-07-22 14:00:00,Time and Materials,1200,"39,000 EUR",41540.0800,0,0.0,180.5,21.50
1,2764,70285,Our biggest project,EXT 6a8e0ca747,Europe,2019-10-01 08:00:00,2021-04-30 11:00:00,Time and Materials,1200,"1,450,999 USD",473002.0800,0,0.0,1241.5,458.25
2,12916,68093,Upgrade to new version,TIM THE LEAVER,Europe,2018-09-28 08:00:00,2020-04-27 17:00:00,Fixed Fee,1200,"127,500 EUR",142411.5533,0,0.0,0.0,0.00
3,12916,68094,Small change on top of upgrade,JOHN THE MANAGER,Europe,2018-09-28 08:00:00,2020-05-11 17:00:00,Time and Materials,1200,"12,500 EUR",0.0000,0,0.0,0.0,0.00
4,18933,68726,Transition from another product,JANE THE MANAGER,Europe,2019-03-01 08:00:00,2020-05-27 16:00:00,Fixed Fee,1200,"143,250 EUR",132677.3616,0,0.0,525.5,5.00


* Timesheets (your team / your projects, note that other teams can contribute to your projects and your team might work in other projects) 
    * Typically arrive monthly or weekly
(optional) Plans
    * If there are no plans, below we talk about an option to “plan auto-fill”

But your ERP probably does not present you anything nicely in terms of parameters you’d want to control on your level  (otherwise you won’t read this anyway). There is even maybe a project running so it will do so by 2025. 
If you are steering this boat long enough – you might have ended up with some quick and dirty calculation, SQL queries to SAP (with field names still in German) or Excel file. Why? Because, well, your bosses don’t give a shit – they already hired you and a better ERP is coming in 2025 anyway – so they want to know how much money your team makes and why is it not so profitable (because it never is)

In [22]:
# show timesheets

timesheet=data["timesheet"]
timesheet.head(n=5)

,Project,Activity,Billing,User,Period Starting,Effort Type,Total (Days),Sun,Mon,Tue,Wed,Thu,Fri,Sat
0,PRJ d41d8cd98f,Consultancy,Billable,EXT 800982ab08,2020-02-02,Actual,2.5,0.0,0.5,0.5,0.5,0.5,0.5,0.0
1,PRJ d41d8cd98f,Consultancy,Billable,EXT c8c92ca432,2020-02-02,Actual,2.5,0.0,0.5,0.5,0.5,0.5,0.5,0.0
2,PRJ d41d8cd98f,Consultancy,Billable,EXT c8c92ca432,2020-02-09,Actual,2.5,0.0,0.5,0.5,0.5,0.5,0.5,0.0
3,PRJ d41d8cd98f,Consultancy,Billable,EXT 800982ab08,2020-02-09,Actual,2.5,0.0,0.5,0.5,0.5,0.5,0.5,0.0
4,PRJ d41d8cd98f,Consultancy,Billable,EXT c8c92ca432,2020-02-16,Actual,2.5,0.0,0.5,0.5,0.5,0.5,0.5,0.0


Next we are going to create a timeseries out of timesheets which is a bit more involved so you can skip it for now and come back to it later, but the points it that at the end you will get a nice monthly pandas Dataframe that looks like this

In [23]:
#collapse

#collapse
dd=[]
timesheet['Period Starting'] = pd.to_datetime(timesheet['Period Starting'])
weekdays = ['Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat'] 
for i, weekday in enumerate(weekdays):
    columns = [col for col in timesheet if col not in weekdays or col==weekday]
    tmp = timesheet[columns].copy() #dmytro todo: port backward
    tmp = tmp.rename(columns={weekday: 'effort'})
    tmp['date'] = tmp['Period Starting']+pd.to_timedelta('{}D'.format(i)) #dmytro todo: check period starting offset logic
    tmp['weekday'] = weekday
    tmp = tmp.drop(['Period Starting','Total (Days)'],axis=1)
    dd +=[tmp]

timeseries = pd.concat(dd,sort=False).sort_values(['User','date'])
timeseries = timeseries.reset_index().drop('index',axis=1)
timeseries['date'] = pd.to_datetime(timeseries['date']).astype('str')
timeseries['month'] = timeseries['date'].str[:7] # dmytro todo: maybe improve date, month stylistics 
timeseries['weekend'] = timeseries['weekday'].apply(lambda x: x in ['Sun','Sat'])
timeseries['workweek'] = timeseries['weekday'].apply(lambda x: x not in ['Sun','Sat'])
#timeseries['month']
#dmytro todo: clean almost null rows

In [24]:
#timeseries_wide = timeseries.groupby(list(timeseries.columns.drop('effort'))[:2])['effort'].sum()
#.unstack('date').reset_index().fillna(0)
#timeseries_wide

In [25]:
#timeseries.groupby('User')['effort'].sum()#.unstack('date').reset_index().fillna(0)

## Profitability

Let me bore you for a minute. You can skip this part if above was self-explanatory to you. Timesheets and projects budgets are simple. You are probably frightened by IFRS revenue recognition and all that – that’s why your CFO has a nice corner office and you don’t, right. Fortunately, there is a simpler thing called Managerial Accounting. 
So, profit is Revenue – Cost and we intend to measure by it and keep it simple. 

**Revenue**: Let’s say that for a Time & Material project you generate as much revenue as you bill (on agreed rate) up to its budget. For a Fixed Fee you split all the revenue per month as per % of effort you spend / are going to spend in the month (changes can be complicated, but they are easy to be managed in the way you want – just go through introduction and wait for the notebook we promised)

In [110]:
# revenue calculation for a person for a month (SUM REV of all timesheet records * corresp project rates)
revenue_df = timeseries[['User','Project','Billing','effort','date','month']].merge(projects[['Project','BASE RATE USD']],how='left',on='Project')
revenue_df['BASE RATE USD'] = revenue_df['BASE RATE USD'].fillna(def_revenue)
# Dmytro check - Max added the billable flag control below
revenue_df = revenue_df.where(revenue_df['Billing']=='Billable').dropna()
revenue_df['daily_revenue'] = revenue_df['BASE RATE USD'] * revenue_df['effort']
revenue = revenue_df.groupby(['User','month'])['daily_revenue'].sum().unstack().fillna(0)
revenue = revenue[revenue.sum(1)>0]
revenue.head()
#revenue_df.effort.sum()
#revenue.shape
#revenue.sum()
#dmytro todo: check null effort, do smart groupby

month,2020-02,2020-03,2020-04
User,,,
CATTY THE NEW MANAGER,11550.00,19350.0,9412.5
EXT 08a7eee3f6,14056.25,4850.0,3612.5
EXT 243a5f992e,7800.00,7500.0,9450.0
EXT 29fc29214e,4350.00,13200.0,3600.0
EXT 2fbb35eab7,24000.00,28480.0,26240.0


**Cost**: So, let’s start with the fact that the average cost of \\$800 (or whatever) is bullshit. You might have people working in different countries and of totally different levels. Talk to your finance team and get some estimates from them (or make it together). We’d say if you were appointed to run a team or business unit – they owe you that much. We will assume below that you managed to get 3 grades (Associate, Base, Senior) per Country you have staff. Though costs per individual will go nicely also. Let's calculate direct cost (here we check the cost of effort posted on timesheets, presuming they are full - can check per number of working days in a month also). We are not interesting by external resources, so we'll filter them out

In [124]:
#headcount.head()

In [125]:
# cost calculation for a project (SUM COST of all timesheet records * corresp cost rates - see roles)
# dmytro todo: apply this to timeseries

cost_df = timeseries[['User','Project','effort','date','month']].merge(headcount[['NAME','EU']],how='left',left_on='User',right_on='NAME')

cost_df['EU'] = cost_df['EU'].fillna(def_cost)
# todo: Dmytro - anything simpler to remove all starting with "EXT"
cost_df = cost_df.where(cost_df['User'].map(lambda x: x[:3]!='EXT')).dropna()
cost_df['daily_cost'] = cost_df['EU'] * cost_df['effort']
cost = cost_df.groupby(['User','month'])['daily_cost'].sum().unstack().fillna(0)
cost = cost[cost.sum(1)>0]
cost.head()
# cost_df.effort.sum()
# cost.shape
# cost.sum()
#dmytro todo: check null effort, do smart groupby

month,2020-02,2020-03,2020-04
User,,,
CATTY THE NEW MANAGER,5362.50,11343.75,9332.8125
FRANK THE CONSTULANT,7700.00,8662.50,7768.7500
IVAN THE PROGRAMMER,11481.25,12925.00,11618.7500
JACK THE EXPERT,18750.00,20350.00,17100.0000
JACK THE MANAGER,11800.00,10900.00,9600.0000


## More compex stuff

Now let's actually show how to apply data science to data you've seen above

Let's start with a chart on how effort is spent

In [108]:
# Effort on the project per month

Indeed, you can make it clickable so you can click on the bar and zoom in (it is more then 10 lines, so we will hide the code)

In [20]:
# (Max) Interactive chart (bar chart effort per month, on click - effort in given month per person)
# Make it working
'''
%matplotlib notebook
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

plt.figure()

# setting colors list for bars depending on value
def get_colors(v):
    colors = []
    for i in range(1992,1996):
        color = "green"
        if (bars.loc[i]["high"] < v):
            color = "blue"
        if (bars.loc[i]["low"] > v):
            color = "red"
        colors.append(color)
    return colors   

# replot. isdef = if it is called before any click
def plt_all(isdef, yvalue):
    if isdef:
        bcolors = "white"
        xtitle = 'Value is'
    else:
        #draw a line and set colors
        plt.plot([1991.5,1995.5], [yvalue,yvalue], color="black",alpha=0.5)
        bcolors = get_colors(yvalue)
        xtitle = '{:.0f} is'.format(yvalue)
    plt.bar(bars.index.tolist(), bars["mean"], color=bcolors, yerr=bars["delta"], width = 1, align='center', alpha=0.5, ecolor='black', capsize=10, edgecolor='black')
    plt.xticks(np.arange(1992, 1996, step=1))
    red_patch = mpatches.Patch(color='red', label='below')
    blue_patch = mpatches.Patch(color='blue', label='above')
    green_patch = mpatches.Patch(color='green', label='inside')
    plt.legend(handles=[red_patch,green_patch,blue_patch],loc='upper left',
           title = xtitle,fontsize=7)
    plt.gca().set_title('Click on the plot to validate confidence intervals')

plt_all(True,0.0)

def onclick(event):
    plt.cla()
    plt_all(False,event.ydata)

plt.gcf().canvas.mpl_connect('button_press_event', onclick)

Ok, as a Team Leader you might not do that. Sorry. Let's get some calculations done. First, let's identify "suspicios" behaviors (e.g. a person who has more then 5 days in a row same effort on a given project and it is not 8 hours) - this is a quick check you do yourself, without asking PMO anything and making this official

In [21]:
# Simple anomalies detection
# TODO: call for action, intro to next paper

## Use cases

Few cases we identified when this data is helpfull

* Decision making - e.g. identify top loss-making projects
* Identify projects which require management attention
* Identify top performers (profit-wise)
* Identify non-billable hours and internal projects
* Identify suspicious behaviours (we will write another post about it)
    * Employee enters fixed hours per project per day over a period
    * Anomaly detection
* Revenue and effort projections
* Capacity calculations: load all data but plans and generate automated projection (effort * person * month / revenue * month)
* Create consolidated analytic on demand for higher level of management (e.g. profitability forecast, revenue forecast, unallocated capacity) in case your ERP will not do anything like that
    * Example: crisis cost saving - if you decrease hours for some contractors, how much cost do you save vs how much revenue to you lose

But, that's not the point. The point is that loading your own data (excel, csv, tex - whatever) is simple and manipulating it is also simple - definetely simpler then doing it in numerous excel files and faster then waiting for PMOs. 

Below code is to be moved inside and removed

# Grouping data